In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**IMPORT LIBRARY**

In [3]:
from keras.models import Sequential,load_model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

**AUGMENTASI WITH IMAGEDATAGENERATOR**

In [5]:
train_generator=ImageDataGenerator(
    rescale=1.255,
    width_shift_range=0.2,
    rotation_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True
)

In [10]:
train_set=train_generator.flow_from_directory(
    '/content/drive/MyDrive/covid_xray/xray/train/',
    target_size=(48,48),
    batch_size=32,
    class_mode='categorical'
)

Found 148 images belonging to 2 classes.


In [12]:
test_generator=ImageDataGenerator(
    rescale=1.255
)

In [14]:
test_set=test_generator.flow_from_directory(
    '/content/drive/MyDrive/covid_xray/xray/test/',
    target_size=(48,48),
    batch_size=32,
    class_mode='categorical'
)

Found 40 images belonging to 2 classes.


**CNN MODEL BUILDING**

In [27]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(48,48,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(2,activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [29]:
history=model.fit(train_set,validation_data=(test_set),epochs=20)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.4529 - loss: 80.4183 - val_accuracy: 0.5000 - val_loss: 38.4120
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 18s 769ms/step - accuracy: 0.4757 - loss: 44.2018 - val_accuracy: 0.8000 - val_loss: 1.0752
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 332ms/step - accuracy: 0.5396 - loss: 7.3434 - val_accuracy: 0.5750 - val_loss: 4.1053
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 331ms/step - accuracy: 0.5878 - loss: 3.8868 - val_accuracy: 0.6250 - val_loss: 0.9661
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 764ms/step - accuracy: 0.5501 - loss: 1.5837 - val_accuracy: 0.8500 - val_loss: 0.3350
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 319ms/step - accuracy: 0.6520 - loss: 0.9712 - val_accuracy: 0.8500 - val_loss: 0.2478
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 480ms/step - accuracy: 0.7177 - loss: 0.9353 - val_accuracy: 0.8500 - val_loss: 0.2587
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 334ms/step - accuracy: 0.7536 - loss: 0.6610 - val_accuracy: 0.7500 - val_l

**SAVE THE MODEL**

In [30]:
model.save('cnn.h5')

**PREDIKSI**

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from google.colab import files
from tensorflow.keras.models import load_model  # Tambahkan import untuk load_model


# Memuat model yang sudah dilatih
model_path = 'cnn.h5'  # Sesuaikan dengan path model Anda
model = load_model(model_path)

uploaded = files.upload()

for filename in uploaded.keys():
    print(f'File {filename} successfully uploaded.')

# Ambil nama file gambar yang diunggah (mungkin perlu disesuaikan sesuai dengan nama file yang Anda unggah)
uploaded_image_path = list(uploaded.keys())[0]

# Load dan praproses gambar
img = image.load_img(uploaded_image_path, target_size=(48, 48))
img_array = image.img_to_array(img)
img_array /= 255.0  # Normalisasi nilai piksel

# Tampilkan gambar yang diuji
plt.imshow(img)
plt.show()

# Ubah gambar menjadi bentuk yang dapat digunakan oleh model
input_image = np.expand_dims(img_array, axis=0)

# Lakukan prediksi
predictions = model.predict(input_image)

# Dapatkan label kelas prediksi
predicted_class = np.argmax(predictions)

# Misalnya, jika Anda memiliki file label seperti label_names
label_names = ['NORMAL','PNEUMONIA']
predicted_label = label_names[predicted_class]

# Tampilkan hasil prediksi
print(f'Predicted Class: {predicted_label}')